# SEP tutorial (with `astropy.io.fits`)
This notebook follows the steps from the SEP tutorial (https://sep.readthedocs.io/en/stable/tutorial.html) but **uses `astropy.io.fits`** to read the example FITS image instead of `fitsio`.

**What this notebook does**
- download the example FITS image used by the SEP tutorial (from the SEP GitHub repository)
- display the image
- estimate and subtract a spatially-varying background using `sep.Background`
- detect sources with `sep.extract`
- overplot object ellipses on the image
- perform simple circular aperture photometry with `sep.sum_circle`

> **Requirements:** `sep`, `astropy`, `numpy`, `matplotlib`. Install with `pip install sep astropy matplotlib`.

> If the SEP example FITS URL has moved, change the `IMAGE_URL` variable in the first code cell to point where the example image is hosted (the SEP GitHub `data` folder contains it).

In [ ]:
# Cell 1: imports and where to download the example FITS image
# Replace IMAGE_URL if SEP moves the example image in their repository.
IMAGE_URL = "https://raw.githubusercontent.com/sep-developers/sep/main/data/image.fits"

import numpy as np
import sep
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from astropy.utils.data import download_file
from astropy.io import fits

# Make plots look reasonable in notebooks
plt.rcParams['figure.figsize'] = (10, 8)

In [ ]:
# Cell 2: download the FITS file and read the image data with astropy
# This uses astropy.utils.data.download_file which caches the file locally.
# If your environment has no internet, put the example 'image.fits' in the notebook folder and set filename to it.
try:
    print("Downloading example FITS from:", IMAGE_URL)
    fname = download_file(IMAGE_URL, cache=True)
    print("Saved to:", fname)
except Exception as e:
    print("Download failed (offline?), trying to use local 'image.fits' if present. Error:", e)
    fname = "image.fits"

# Read image data (primary HDU)
data = fits.getdata(fname)
# Astropy returns big-endian arrays for FITS by default; SEP prefers native byte order.
# Ensure native byte order (see SEP docs for details).
if data.dtype.byteorder == '>' or (data.dtype.byteorder == '=' and not np.little_endian):
    data = data.byteswap().newbyteorder()

print("data shape:", data.shape, "dtype:", data.dtype)

In [ ]:
# Cell 3: display the raw image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar(label='pixel value')
plt.title('Raw image (display)')

In [ ]:
# Cell 4: estimate and display the spatially varying background and RMS with SEP
# The Background object internally computes a background map (bkg.back()) and an RMS map (bkg.rms()).
bkg = sep.Background(data)    # default box size is usually fine for the small example image
print("global background (mean):", bkg.globalback)
print("global RMS:", bkg.globalrms)

bkg_image = bkg.back()
bkg_rms = bkg.rms()

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].imshow(bkg_image, interpolation='nearest', origin='lower', cmap='gray')
axes[0].set_title('Background map (bkg.back())')
axes[0].axis('off')
axes[1].imshow(bkg_rms, interpolation='nearest', origin='lower', cmap='gray')
axes[1].set_title('Background RMS map (bkg.rms())')
axes[1].axis('off')

In [ ]:
# Cell 5: subtract the background (to create a background-subtracted image)
# You can do this by: data_sub = data - bkg.back()  OR use the Background object directly in some SEP calls.
data_sub = data - bkg
m, s = np.mean(data_sub), np.std(data_sub)
plt.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar(label='pixel value (background subtracted)')
plt.title('Background-subtracted image')

In [ ]:
# Cell 6: detect objects using sep.extract
# We use a threshold of 1.5 * global RMS (same as the SEP tutorial).
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
print("Number of objects detected:", len(objects))
print("Fields available in objects.dtype.names:", objects.dtype.names)

In [ ]:
# Cell 7: over-plot elliptical approximations for the detected objects
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
ax.set_title('Detected objects (ellipses)')

In [ ]:
# Cell 8: perform circular aperture photometry at the detected object centroids
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)
# show first 10 fluxes
for i in range(min(10, len(flux))):
    print(f"object {i}: flux = {flux[i]:.6f} +/- {fluxerr[i]:.6f}  flag={flag[i]}")

## Notes and troubleshooting
- If you see a `ValueError` about non-native byte order when using `astropy.io.fits`, make the array native byte order as shown in the notebook.
- If the `download_file` step fails (internet blocked), place `image.fits` in the same folder as the notebook and set `fname = 'image.fits'`.\n\n
### Where did I get the example FITS file?
The SEP tutorial's example FITS is included in the SEP repository under the `data/` directory. The notebook attempts to download it from the SEP GitHub repo (see the `IMAGE_URL` variable in the first code cell). If you prefer, manually download the file from the SEP repo and run the notebook locally.\n\n
### Running the notebook
1. Save and open this notebook in Jupyter or JupyterLab.\n2. Install dependencies if needed: `pip install sep astropy matplotlib`.\n3. Run the cells in order.\n\nIf you'd like, I can also:\n- run a version of this notebook here (if the execution environment allowed internet and the `sep` package),\n- or produce a more compact script version (.py) that does the same steps.